<a href="https://colab.research.google.com/github/loicbi/Databricks/blob/develop/16_Databricks_%7C_Spark_%7C_Pyspark_%7C_Bad_Records_Handling_%7C_Permissive%3BDropMalformed%3BFailFast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install findspark

In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=a788732927f78bb904db912ff399cc4e2426ab5e111d2369748946abeebb03aa
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [6]:
import findspark
from pyspark.sql.functions import  * # to_date, col, split, char_length, lit, trim
# findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.types import *
# Create a SparkSession
spark = SparkSession.builder.getOrCreate()
# spark = SparkSession.builder.appName("app_name").getOrCreate()
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

schema = StructType([
    StructField("title", StringType(), True),
    StructField("cast", StringType(), True),
    StructField("country", StringType(), True),
    StructField("date_added", StringType(), True),
    StructField("release_year", IntegerType(), True),
    StructField("rating", StringType(), True),
    StructField("duration", StringType(), True),
    StructField("listed_in", StringType(), True),
    StructField("description", StringType(), True),
    StructField("_corrupt_record", StringType(), True)
])

# df = spark.read.format('csv').option('header', True)\
#           .option('mode', 'PERMISSIVE')\
df_permissive = spark.read.format('csv').option('header', True)\
          .option('mode', 'PERMISSIVE')\
          .schema(schema)\
          .load('/content/drive/MyDrive/DATA_ENGINEER/DATABRICKS/AAAA_________IAM___CODING_____Colab Notebooks/Learning/datasset/AFCON/csv_files/netflix_shows.csv')

df_permissive.printSchema()
df_permissive.show(10)


root
 |-- title: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)
 |-- _corrupt_record: string (nullable = true)

+--------------------+--------------------+-------------+----------+------------+------+---------+--------------------+--------------------+--------------------+
|               title|                cast|      country|date_added|release_year|rating| duration|           listed_in|         description|     _corrupt_record|
+--------------------+--------------------+-------------+----------+------------+------+---------+--------------------+--------------------+--------------------+
|       Blood & Water|Ama Qamata, Khosi...| South Africa| 24-Sep-21|        2021| TV-MA|2 Seasons|In

[Differences between FAILFAST, PERMISSIVE and DROPMALFORED modes in Spark Dataframes
](https://medium.com/@coffeeandtips.tech/differences-between-failfast-permissive-and-dropmalfored-modes-in-spark-dataframes-f803ddb098f0)

## Dataframe result using DROPMALFORMED mode



In [7]:
df_dropmalformed =  spark.read.format('csv').option('header', True)\
          .option('mode', 'DROPMALFORMED')\
          .schema(schema)\
          .load('/content/drive/MyDrive/DATA_ENGINEER/DATABRICKS/AAAA_________IAM___CODING_____Colab Notebooks/Learning/datasset/AFCON/csv_files/netflix_shows.csv')
df_dropmalformed.show(10)

+--------------------+--------------------+-------------+----------+------------+------+---------+--------------------+--------------------+---------------+
|               title|                cast|      country|date_added|release_year|rating| duration|           listed_in|         description|_corrupt_record|
+--------------------+--------------------+-------------+----------+------------+------+---------+--------------------+--------------------+---------------+
|       Blood & Water|Ama Qamata, Khosi...| South Africa| 24-Sep-21|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|           NULL|
|           Ganglands|Sami Bouajila, Tr...|       France| 24-Sep-21|        2021| TV-MA| 1 Season|Crime TV Shows, I...|To protect his fa...|           NULL|
|Jailbirds New Orl...|                NULL|United States| 24-Sep-21|        2021| TV-MA| 1 Season|Docuseries, Reali...|Feuds, flirtation...|           NULL|
|       Midnight Mass|Kate Siegel, Zach...|United States| 

## Dataframe result using FAILFAST mode



In [8]:
df_failfast =  spark.read.format('csv').option('header', True)\
          .option('mode', 'FAILFAST')\
          .schema(schema)\
          .load('/content/drive/MyDrive/DATA_ENGINEER/DATABRICKS/AAAA_________IAM___CODING_____Colab Notebooks/Learning/datasset/AFCON/csv_files/netflix_shows.csv')
df_failfast.show(10)

Py4JJavaError: An error occurred while calling o143.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 3) (dd9e6150e206 executor driver): org.apache.spark.SparkException: [MALFORMED_RECORD_IN_PARSING.WITHOUT_SUGGESTION] Malformed records are detected in record parsing: [Kota Factory,Mayur More, Jitendra Kumar, Ranjan Raj, Alam Khan, Ahsaas Channa, Revathi Pillai, Urvi Singh, Arun Kumar,India,24-Sep-21,null,TV-MA,2 Seasons,International TV Shows, Romantic TV Shows, TV Comedies,In a city of coaching centers known to train India’s finest collegiate minds, an earnest but unexceptional student and his friends navigate campus life.,Kota Factory,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam Khan, Ahsaas Channa, Revathi Pillai, Urvi Singh, Arun Kumar",India,24-Sep-21,QQQQAAAAA,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV Comedies","In a city of coaching cente